# Region Logic
The goal of this nb is to figure out some logic for handling regions.
* Figure out how routes are made and returned by mapbox
* Figure out how to break a route down into it's states and mileage
* create region constants by making a list of the state names (or coords, or which ever works best with mapbox)

In [1]:
import requests
from dotenv import load_dotenv
load_dotenv()
import os

token = os.environ.get('MAPBOX_TOKEN')

url = 'https://api.mapbox.com/directions/v5/mapbox/driving?access_token='
url += token
body = {'coordinates': '-122.3321,47.6062;-122.4443,47.2529;-122.6750,45.5051'}
#       'steps': 'true'}
#       'annotations': 'distance'}

resp = requests.post(url, data = body)
resp

<Response [200]>

In [2]:
resp.request.body

'coordinates=-122.3321%2C47.6062%3B-122.4443%2C47.2529%3B-122.6750%2C45.5051'

In [3]:
resp.request.headers

{'User-Agent': 'python-requests/2.25.0', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'Content-Length': '75', 'Content-Type': 'application/x-www-form-urlencoded'}

In [4]:
resp.json()

{'routes': [{'weight_name': 'auto',
   'weight': 12313.017,
   'duration': 11837.105,
   'distance': 297008.094,
   'legs': [{'steps': [],
     'admins': [{'iso_3166_1_alpha3': 'USA', 'iso_3166_1': 'US'}],
     'duration': 2364.449,
     'distance': 54183.391,
     'weight': 2789.213,
     'summary': 'I 5, I 5 South'},
    {'steps': [],
     'admins': [{'iso_3166_1_alpha3': 'USA', 'iso_3166_1': 'US'},
      {'iso_3166_1_alpha3': 'USA', 'iso_3166_1': 'US'}],
     'duration': 9472.656,
     'distance': 242824.703,
     'weight': 9523.804,
     'summary': 'I 5 South, I 405 South'}],
   'geometry': 'mbqaHz~siVMLx@jCmCdC{D_Mf{@u|@lAa@|HgAfDIlt@{AfjXi~Itrc@rnIpkDbcCha@jnOjIzXf@`Gq[pUgXfKcYpEaR~DqAPmPxBzCpl@WDcKtAyCgl@tIwFlfBiZdg@nfAt{L|wF~pH|`P~dPpi`Azek@drSvrJq}EhsN|M`dQ{sRl_o@xnFroKw`EvfWh{AhvXubJzkXwdYd`ZyaIhib@fkCzQcDn_Alq@lwKoOuAvUsAfBMaKZjPhGmq@}sDmk@iU|A_R~F'}],
 'waypoints': [{'distance': 12.556,
   'name': '',
   'location': [-122.332143, 47.606309]},
  {'distance': 12.64,
   'name'

In [5]:
# Okay I know how to get routes, now lets see if I can decode coords to states

url2 = 'https://api.mapbox.com/geocoding/v5/mapbox.places/-122.3321,47.6062.json?access_token='
url2 += token
resp2 = requests.get(url2)
resp2

<Response [200]>

In [6]:
resp2.json()

{'type': 'FeatureCollection',
 'query': [-122.3321, 47.6062],
 'features': [{'id': 'poi.137439024267',
   'type': 'Feature',
   'place_type': ['poi'],
   'relevance': 1,
   'properties': {'foursquare': '552e1a97498eec260fd37d3f',
    'landmark': True,
    'address': '2738 Alki Ave SW',
    'category': 'beauty, hair, salon, barber, hairdresser'},
   'text': "Tiffany's Hair Design",
   'place_name': "Tiffany's Hair Design, 2738 Alki Ave SW, Seattle, Washington 98164, United States",
   'center': [-122.33207, 47.60621],
   'geometry': {'coordinates': [-122.33207, 47.60621], 'type': 'Point'},
   'context': [{'id': 'neighborhood.2101930', 'text': 'Downtown'},
    {'id': 'postcode.10427398245565200', 'text': '98164'},
    {'id': 'place.11115494111229470', 'wikidata': 'Q5083', 'text': 'Seattle'},
    {'id': 'region.9713796497246050',
     'wikidata': 'Q1223',
     'short_code': 'US-WA',
     'text': 'Washington'},
    {'id': 'country.19678805456372290',
     'wikidata': 'Q30',
     'short_cod

In [7]:
resp2.json()['features'][4]['text']

'Washington'

## Summary
* coordinates need to be a string formated as '_long,lat;long,lat_'. 
* annotations - distance is overkill for now, the functional distance between legs is packed into the 'legs' key
* step.maneuver.location is likely the best way for me to find when travelers change regions
* I can reverse look up a geocode using the mapbox.places endpoint
* I might need to loop through the features to find the 'region' feature

## mvp route -> regions workflow
A basic workflow for going from a route, to measuring the travel distance in each region. No working predition models, just proof of workflow.

In [8]:
# First lets define our PADD regions
padds = {'1a': 
             ['Maine', 'New Hampshire', 'Vermont', 'Massachusetts', 
             'Connecticut', 'Rhode Island'],
         '1b':
             ['New York', 'New Jersey', 'Pennsylvania', 'Deleware', 'Maryland'],
         '1c':
             ['West Virginia', 'Virginia', 'North Carolina', 'South Carolina',
             'Georgia', 'Florida'],
         '2':
             ['North Dakota', 'South Dakota', 'Nebraska', 'Kansas', 'Oklahoma',
             'Minnesota', 'Iowa', 'Missouri', 'Wisconsin', 'Illinois', 
             'Tennessee', 'Kentucky', 'Indiana', 'Ohio', 'Michigan'],
         '3':
             ['New Mexico', 'Texas', 'Arkansas', 'Louisiana', 'Mississippi', 
             'Alabama'],
         '4':
             ['Idaho', 'Utah', 'Montana', 'Wyoming', 'Colorado'],
         '5':
             ['Washington', 'Oregon', 'California', 'Nevada', 'Arizona', 
             'Alaska', 'Hawaii']
        }

# And our map of models
gas_models = {'1a': 1.00, '1b': 1.50, '1c': 2.00, '2': 2.50, '3': 3.00, '4': 3.50, '5': 4.00}

In [9]:
# lets make a function for checking coords for their state
def coord_to_state(coord):
    base_url = 'https://api.mapbox.com/geocoding/v5/mapbox.places/'
    token = os.environ.get('MAPBOX_TOKEN')
    constructed_url = base_url + str(coord[0]) + ',' + str(coord[1]) + '.json?access_token=' + token
    
    # search for the region feature
    resp = requests.get(constructed_url).json()['features']
    
    for feature in resp:
        if 'region' in feature['place_type']:
            return feature['text']
    
    return 'state not found'

In [10]:
# Testing the function works
seattle = [-122.3321,47.6062]
boise = [-116.2023,43.6150]
vegas = [-115.1398, 36.1699]

print(coord_to_state(seattle))
print(coord_to_state(boise))
print(coord_to_state(vegas))

Washington
Idaho
Nevada


In [11]:
# Now lets make a function that returns a region code
def coord_to_region(coord):
    #TODO consider moving padds map here vs keeping it global in our api
    state = coord_to_state(coord)
    for key in padds:
        if state in padds[key]:
            return key
        
    return 'Not in padds'

In [12]:
# Testing out the coord to region, should give 5, 4, 5.
print(coord_to_region(seattle))
print(coord_to_region(boise))
print(coord_to_region(vegas))

5
4
5


In [13]:
def region_gas_predictions(region, date = None):
    # TODO this needs to be populated with region models. Dummy code for now
    # consider keeping the models in global space to improve response time
    
    return gas_models[region]

In [14]:
# Should generate $400
100 * region_gas_predictions(coord_to_region(seattle))

400.0

In [15]:
# Lets do an example route.  Seattle WA, to Boise ID, to Las Vegas NV
route = {'coordinates': '-122.3321,47.6062;-116.2023,43.6150;-115.1398, 36.1699',
        'steps': 'true'}

trip = requests.post(url, data = route)
trip

<Response [200]>

In [16]:
# I will need to iterate along each leg of the first route recommended
# commented out because it is huge
#trip.json()['routes'][0]['legs']

# for getting the end location of a maneuvor
trip.json()['routes'][0]['legs'][0]['steps'][0]['intersections'][-1]['location']

[-122.332151, 47.606318]

In [17]:
# for pulling the distance out of a step
trip.json()['routes'][0]['legs'][0]['steps'][0]['distance']

9

In [18]:
distance = 0
prev_reg = 0
cur_reg = 0
reg_dist_map = {'distances': [], 'regions': []}
# loop each leg
for leg in trip.json()['routes'][0]['legs']:
    # loop each step in the leg
    for step in leg['steps']:
        prev_reg = cur_reg
        
        # collect distance traveled
        distance += step['distance']
        
        # check the end location of each step.manuever
        # TODO is 'intersections' really the appropriate place to look?
        # explore some more
        coords = step['intersections'][-1]['location']
        
        cur_reg = coord_to_region(coords)
#         if cur_reg == 'Not in padds':
#             print(coord_to_state(coords))
#             print(coords)
        
        if prev_reg == 0:
            prev_reg = cur_reg
        elif cur_reg != prev_reg:
            reg_dist_map['distances'].append(distance)
            distance = 0
            reg_dist_map['regions'].append(prev_reg)

reg_dist_map['distances'].append(distance)
reg_dist_map['regions'].append(prev_reg)
        
    
# multiple the mileage by the region rate
reg_dist_map

{'distances': [805365.605, 525191.9, 488783.43799999997],
 'regions': ['5', '4', '5']}

In [19]:
trip.json()['routes'][0]['distance']

1819340.875

In [20]:
# should be comparable
sum(reg_dist_map['distances'])

1819340.943

In [21]:
#Then I can make some assumptions and calculate cost by region!
# I should get approximately $161.436055202

mpg = 27
meter_to_mile = 0.000621371
total = 0

for i, distance in enumerate(reg_dist_map['distances']):
    
    total += ((distance * meter_to_mile) / mpg) * region_gas_predictions(reg_dist_map['regions'][i])
    
total

161.4360480119986

# Conclusion
For a proof of workflow this is good. Outstanding things to make this a perfect bundle to deploy on the api.
* Functionalize the last few cells
* either pickle your gas prediction models
* or make end points to calculate your models that can be run on a schedule
* connect the models to the region logic